# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [ ]:
base_dir = r'./data/raw'
folders = os.listdir(base_dir)
print(folders)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

datagenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train_generator = datagenerator.flow_from_directory(
    base_dir,
    target_size=(250 , 250),
    batch_size=32,
    subset='training',
)

test_generator = datagenerator.flow_from_directory(
    base_dir,
    target_size=(250 , 250),
    batch_size=32,
    subset='validation',
)

In [ ]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(patience=3)
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_acc',
    patience= 2,
    verbose=1,
)

callbacks = [earlystop, learning_rate_reduction]

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,Dropout

inputs = Input(shape=(250, 250, 3))

x = Conv2D(32, (5,5), input_shape = (250 , 250 , 3), activation = 'relu')(inputs)
x = MaxPooling2D(3,3)(x)
x = Conv2D(64, (5,5) , activation = 'relu')(x)
x = MaxPooling2D((3,3))(x)
x = Conv2D(128 , (5,5) , activation = 'relu')(x)
x = MaxPooling2D(3,3)(x)
x = Flatten()(x)
x = Dropout(0.5, seed = 5)(x)
x = Dense(128 , activation = 'relu')(x)

outputs = Dense(4, activation = 'softmax')(x)

model:Model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="Adam", loss="mean_squared_error", metrics=["accuracy"])
model.fit(
    train_generator, 
    epochs = 10,
    steps_per_epoch = train_generator.n // train_generator.batch_size,
    validation_data = test_generator,
    validation_steps = test_generator.n // test_generator.batch_size,
    callbacks= callbacks
)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot(title='Model')

In [ ]:
accurracies = pd.DataFrame(model.history.history)
accurracies[['accuracy','val_accuracy']].plot(title='Model')